In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load saved model
model = tf.keras.models.load_model("mnist_model.h5")

# Prediction function
def predict_digit(img):
    img = cv2.resize(img, (28, 28))
    img = img.reshape(1, 28, 28, 1)
    img = img.astype("float32") / 255.0
    pred = model.predict(img)
    return np.argmax(pred), np.max(pred)

# Start camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Frame shape
    h, w, _ = frame.shape

    # ROI (blue box)
    box_size = 250
    x1 = w // 2 - box_size // 2
    y1 = h // 2 - box_size // 2
    x2 = w // 2 + box_size // 2
    y2 = h // 2 + box_size // 2

    # Rectangle
    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

    # ROI
    roi = frame[y1:y2, x1:x2]

    # Grayscale + Gaussian Blur + Threshold
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thres = cv2.threshold(blur, 128, 255, cv2.THRESH_BINARY_INV)

    # Prediction
    digit, acc = predict_digit(thres)

    # Prediction
    cv2.putText(frame, f"Prediction: {digit} ({acc*100:.2f}%)",
                (10, 50), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 255, 0), 2)

    # Original frame
    cv2.imshow("Digit Recognition", frame)

    # processed (B/W)
    cv2.imshow("Processed ROI", thres)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
